1. 필요한 라이브러리 임포트

In [77]:
# Cell 1: Import Libraries
import cv2
import mediapipe as mp
import numpy as np
import time
import os
import sys # sys.exit() 대신 메시지 출력 후 종료를 위해 사용될 수 있습니다.

2. 전역 설정 및 MediaPipe 모델, 웹캠 초기화

In [78]:
# Cell 2: Global Setup and Initialization

# MediaPipe Hands 모델 초기화
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=2, # 한 번에 감지할 최대 손 개수 (기본값 2개)
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# 웹캠 초기화 (한 번만 실행)
cap = cv2.VideoCapture(0)

# 웹캠이 제대로 열렸는지 확인
if not cap.isOpened():
    print("오류: 웹캠을 열 수 없습니다. 웹캠이 연결되어 있고 다른 프로그램에서 사용 중이 아닌지 확인하세요.")
    print("이후 셀에서 카메라를 사용하는 함수는 작동하지 않을 수 있습니다.")
    # Jupyter 환경이므로 sys.exit() 대신 메시지 출력 후 진행
else:
    print("웹캠이 성공적으로 초기화되었습니다.")

# 데이터셋을 저장할 디렉토리 생성
dataset_dir = 'test_dataset'
os.makedirs(dataset_dir, exist_ok=True)
print(f"데이터셋 저장 디렉토리: '{dataset_dir}'")

웹캠이 성공적으로 초기화되었습니다.
데이터셋 저장 디렉토리: 'test_dataset'


3. 제스처 데이터 수집 함수 정의

In [79]:
# Cell 3: Define Gesture Data Collection Function (for Two Hands)

def collect_gesture_data(gesture_name: str, gesture_label_id: int,
                         secs_for_action: int = 30, seq_length: int = 30):
    """
    웹캠을 통해 특정 제스처의 데이터를 수집하고 NumPy 파일로 저장합니다.
    두 손을 감지하도록 설정되어 있으며, 두 손의 랜드마크 및 각도 데이터를 하나의 프레임으로 저장합니다.
    두 손이 모두 감지되지 않으면 해당 프레임은 건너뛸 수 있습니다.

    Args:
        gesture_name (str): 수집할 제스처의 이름 (예: 'come', 'away', 'spin').
        gesture_label_id (int): 해당 제스처에 할당될 숫자 라벨 ID (예: 0, 1, 2).
        secs_for_action (int, optional): 각 제스처에 대해 데이터를 수집할 시간(초). 기본값은 30초.
        seq_length (int, optional): 시퀀스 데이터 생성 시 하나의 시퀀스에 포함될 프레임 길이. 기본값은 30.
    """
    if not cap.isOpened():
        print(f"경고: 웹캠이 열려 있지 않습니다. '{gesture_name}' 데이터 수집을 건너뜜니다.")
        return

    data = [] # 현재 제스처의 원시 데이터를 저장할 리스트
    created_time = int(time.time()) # 현재 데이터 수집 세션의 고유 타임스탬프

    # 데이터 수집 시작 전 사용자에게 준비 시간 제공
    print(f"\n--- '{gesture_name.upper()}' 동작 데이터 수집 준비 ---")
    print("카메라를 향해 두 손을 준비해주세요.")

    ret, img = cap.read()
    if not ret:
        print("오류: 웹캠에서 프레임을 읽을 수 없습니다. 카메라 연결을 확인하세요.")
        return

    img = cv2.flip(img, 1) # 좌우 반전 (거울 모드)
    # 수집 대기 메시지 표시
    cv2.putText(img, f'Ready for {gesture_name.upper()} in 3 seconds...', org=(10, 30),
                fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0, 255, 0), thickness=2)
    cv2.imshow('Gesture Data Collection (Two Hands)', img) # 창 이름 변경
    cv2.waitKey(3000) # 3초 대기

    start_time = time.time()
    print(f"'{gesture_name.upper()}' 동작 데이터 수집 시작! {secs_for_action}초 동안 동작을 수행해주세요.")

    # 지정된 시간 동안 데이터 수집 루프
    while time.time() - start_time < secs_for_action:
        ret, img = cap.read()
        if not ret:
            print("오류: 웹캠에서 프레임을 읽을 수 없습니다. 카메라 연결을 확인하세요.")
            break

        img = cv2.flip(img, 1) # 좌우 반전
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # MediaPipe 처리를 위해 BGR -> RGB 변환
        result = hands.process(img) # MediaPipe를 이용한 손 랜드마크 감지
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) # OpenCV 표시를 위해 RGB -> BGR 변환

        if result.multi_hand_landmarks and len(result.multi_hand_landmarks) == 2:
            # ***두 손이 모두 감지된 경우에만 데이터를 처리***
            # 만약 한 손만 감지되더라도 처리하고 싶다면, 이 조건을 제거하고
            # 각 손의 데이터를 빈 배열로 채우는 로직을 추가해야 합니다.
            # (예: np.zeros(size)를 사용하여 감지되지 않은 손의 데이터를 채움)

            # 왼쪽/오른쪽 손을 식별하고 순서를 맞추는 로직 (선택 사항, 필요 시 추가)
            # 여기서는 단순히 감지된 두 손을 순서대로 처리합니다.
            # 랜드마크 0번(손목)의 x좌표를 기준으로 왼쪽/오른쪽을 구분할 수 있습니다.
            handedness = [(lm.landmark[0].x, i) for i, lm in enumerate(result.multi_hand_landmarks)]
            handedness.sort() # x좌표를 기준으로 정렬 (왼쪽 손부터)

            combined_hand_data = []

            for _, hand_idx in handedness: # 정렬된 순서로 각 손 처리
                res = result.multi_hand_landmarks[hand_idx]

                # 21개 손 랜드마크의 x, y, z 좌표 및 가시성(visibility) 추출
                joint = np.zeros((21, 4))
                for j, lm in enumerate(res.landmark):
                    joint[j] = [lm.x, lm.y, lm.z, lm.visibility]

                # 관절 사이의 벡터 계산 (부모-자식 관절 쌍)
                v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
                v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
                v = v2 - v1 # 벡터 (20, 3)

                # 벡터 정규화 (길이를 1로 만듦)
                norm_v = np.linalg.norm(v, axis=1)
                v = v / (norm_v[:, np.newaxis] + 1e-8) # 0으로 나누는 오류 방지

                # 두 벡터 사이의 각도 계산 (내적을 이용)
                dot_product = np.einsum('nt,nt->n',
                                         v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:],
                                         v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])
                dot_product = np.clip(dot_product, -1.0, 1.0) # arccos의 도메인 클리핑

                angle = np.arccos(dot_product) # 라디안 각도 (15,)
                angle = np.degrees(angle) # 라디안을 도로 변환

                # 각 손의 랜드마크 및 각도 데이터 추가
                # 두 손의 데이터를 하나의 벡터로 만듭니다.
                # (21 * 4) 랜드마크 + 15 각도 = 84 + 15 = 99
                combined_hand_data.append(joint.flatten()) # 손 랜드마크 84개
                combined_hand_data.append(angle)           # 손 각도 15개

                mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS) # 화면에 그리기

            # 두 손의 데이터(랜드마크 + 각도 + 랜드마크 + 각도)에 제스처 라벨 ID 추가
            # 최종 데이터 구조: [손1_랜드마크(84), 손1_각도(15), 손2_랜드마크(84), 손2_각도(15), 제스처_ID(1)]
            # 총 84 + 15 + 84 + 15 + 1 = 199개 데이터 포인트
            d = np.concatenate(combined_hand_data + [np.array([gesture_label_id])]).astype(np.float32)
            data.append(d) # 수집된 데이터를 리스트에 추가
        else:
            # 두 손이 모두 감지되지 않은 프레임은 건너뜁니다.
            # 필요에 따라 이곳에 빈 데이터를 추가하거나 경고를 출력할 수 있습니다.
            pass # print("경고: 두 손이 모두 감지되지 않았습니다.")


        # 화면에 현재 동작 및 남은 시간 표시
        elapsed_time = int(time.time() - start_time)
        remaining_time = max(0, secs_for_action - elapsed_time)
        cv2.putText(img, f'{gesture_name.upper()}: {remaining_time}s left (Two Hands)', org=(10, 30),
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
        cv2.imshow('Gesture Data Collection (Two Hands)', img)

        # 'q' 키를 누르면 수집 중단
        if cv2.waitKey(1) == ord('q'):
            print("사용자 요청으로 데이터 수집을 중단합니다.")
            break # 내부 루프 종료

    # 데이터 저장
    if len(data) > 0:
        data = np.array(data)
        print(f"'{gesture_name}' raw 데이터 shape: {data.shape}")
        raw_filename = os.path.join(dataset_dir, f'raw_{gesture_name}_{created_time}.npy')
        np.save(raw_filename, data)
        print(f"'{gesture_name}' raw 데이터가 저장되었습니다: {raw_filename}")

        # 시퀀스 데이터 생성 및 저장
        full_seq_data = []
        if len(data) >= seq_length:
            for seq in range(len(data) - seq_length + 1):
                full_seq_data.append(data[seq:seq + seq_length])

        if len(full_seq_data) > 0:
            full_seq_data = np.array(full_seq_data)
            print(f"'{gesture_name}' sequence 데이터 shape: {full_seq_data.shape}")
            seq_filename = os.path.join(dataset_dir, f'seq_{gesture_name}_{created_time}.npy')
            np.save(seq_filename, full_seq_data)
            print(f"'{gesture_name}' sequence 데이터가 저장되었습니다: {seq_filename}")
        else:
            print(f"경고: '{gesture_name}' 시퀀스 데이터를 생성할 충분한 데이터가 없습니다.")
            print(f"  (수집된 프레임: {len(data)}, 필요한 시퀀스 길이: {seq_length})")
    else:
        print(f"'{gesture_name}'에 대해 수집된 데이터가 없습니다.")

    print(f"--- '{gesture_name.upper()}' 데이터 수집 완료 ---")

4. 함수 사용 예시 (데이터 수집 실행)

In [ ]:
# Cell 4: Example Usage - Collect Data for Defined Gestures

# 수집할 제스처 목록과 해당 라벨 ID 정의
# 모델 학습 시 이 라벨 ID를 사용하여 각 제스처를 분류합니다.
gestures_to_collect = [
    {'name': 'flower', 'label_id': 0},
    {'name': 'crown', 'label_id': 1},
    {'name': 'heartbeat', 'label_id': 2},
    {'name': 'firework', 'label_id': 3},
    {'name': 'bear', 'label_id': 4},
    {'name': 'cat', 'label_id': 5},
    {'name': 'gun', 'label_id': 6},
    {'name': 'heart_on_the_cheek', 'label_id': 7},
    {'name': 'son_celebration', 'label_id': 8},
    {'name': 'tiger', 'label_id': 9},
    {'name': 'pipe', 'label_id': 10},
    {'name': 'landmarks', 'label_id': 11},
    #{'name': 'sumi', 'label_id': 12},
    # 필요한 경우 여기에 더 많은 제스처를 추가하세요.
]

# 각 제스처에 대해 데이터 수집 함수 호출
for gesture_info in gestures_to_collect:
    collect_gesture_data(
        gesture_name=gesture_info['name'],
        gesture_label_id=gesture_info['label_id'],
        secs_for_action=60, # 각 제스처당 30초 수집
        seq_length=30      # 시퀀스 길이 30프레임
    )

print("\n모든 지정된 제스처에 대한 데이터 수집이 완료되었습니다.")

cv2.destroyAllWindows()


--- 'FLOWER' 동작 데이터 수집 준비 ---
카메라를 향해 두 손을 준비해주세요.
'FLOWER' 동작 데이터 수집 시작! 60초 동안 동작을 수행해주세요.
'flower' raw 데이터 shape: (1055, 199)
'flower' raw 데이터가 저장되었습니다: test_dataset\raw_flower_1750301079.npy
'flower' sequence 데이터 shape: (1026, 30, 199)
'flower' sequence 데이터가 저장되었습니다: test_dataset\seq_flower_1750301079.npy
--- 'FLOWER' 데이터 수집 완료 ---

--- 'CROWN' 동작 데이터 수집 준비 ---
카메라를 향해 두 손을 준비해주세요.
'CROWN' 동작 데이터 수집 시작! 60초 동안 동작을 수행해주세요.
'crown' raw 데이터 shape: (631, 199)
'crown' raw 데이터가 저장되었습니다: test_dataset\raw_crown_1750301143.npy
'crown' sequence 데이터 shape: (602, 30, 199)
'crown' sequence 데이터가 저장되었습니다: test_dataset\seq_crown_1750301143.npy
--- 'CROWN' 데이터 수집 완료 ---

--- 'HEARTBEAT' 동작 데이터 수집 준비 ---
카메라를 향해 두 손을 준비해주세요.
'HEARTBEAT' 동작 데이터 수집 시작! 60초 동안 동작을 수행해주세요.
'heartbeat' raw 데이터 shape: (1036, 199)
'heartbeat' raw 데이터가 저장되었습니다: test_dataset\raw_heartbeat_1750301206.npy
'heartbeat' sequence 데이터 shape: (1007, 30, 199)
'heartbeat' sequence 데이터가 저장되었습니다: test_dataset\seq_heartbeat_1750301206.n

5. 리소스 정리

In [81]:
# Cell 5: Cleanup Resources

# 웹캠 리소스 해제
if cap.isOpened():
    cap.release()
    print("웹캠 리소스가 해제되었습니다.")

# 모든 OpenCV 창 닫기
cv2.destroyAllWindows()
print("모든 OpenCV 창이 닫혔습니다.")

웹캠 리소스가 해제되었습니다.
모든 OpenCV 창이 닫혔습니다.
